## Day 27 Lecture 1 Assignment

In this assignment, we will learn statistical significance in linear models. We will use the google play store dataset loaded below and analyze the regression from this dataset.

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor


from scipy import stats
from scipy.stats import bartlett
from scipy.stats import levene


import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
reviews = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/googleplaystore.csv"
)

<IPython.core.display.Javascript object>

In [4]:
reviews.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


<IPython.core.display.Javascript object>

We will predict app ratings using other features describing the app. To use these features, we must clean the data.

To simplify, we will remove the app, category, size, installs, genres, last updated, current ver, and android ver columns. 

In [5]:
# answer below:
reviews = reviews.drop(
    columns=[
        "App",
        "Category",
        "Size",
        "Installs",
        "Genres",
        "Last Updated",
        "Current Ver",
        "Android Ver",
    ]
)

<IPython.core.display.Javascript object>

In [6]:
reviews

,Rating,Reviews,Type,Price,Content Rating
0,4.1,159,Free,0,Everyone
1,3.9,967,Free,0,Everyone
2,4.7,87510,Free,0,Everyone
3,4.5,215644,Free,0,Teen
4,4.3,967,Free,0,Everyone
...,...,...,...,...,...
10836,4.5,38,Free,0,Everyone
10837,5.0,4,Free,0,Everyone
10838,NaN,3,Free,0,Everyone
10839,4.5,114,Free,0,Mature 17+


<IPython.core.display.Javascript object>

Check for missing values and remove all rows containing missing values

In [7]:
# answer below:
reviews.isna().mean()
reviews = reviews.dropna()
reviews.isna().mean()
reviews.shape

(9366, 5)

<IPython.core.display.Javascript object>

Remove outliers from the Type and Content Rating columns (very rare values that won't train well).

In [8]:
reviews["Type"].value_counts()

#reviews["Content Rating"].value_counts()


Free    8719
Paid     647
Name: Type, dtype: int64

<IPython.core.display.Javascript object>

In [9]:
# answer below:
reviews = reviews[reviews["Content Rating"] != "Adults only 18+"]
reviews = reviews[reviews["Content Rating"] != "Unrated"]


<IPython.core.display.Javascript object>

In [10]:
reviews["Content Rating"].value_counts()

Everyone        7420
Teen            1084
Mature 17+       461
Everyone 10+     397
Name: Content Rating, dtype: int64

<IPython.core.display.Javascript object>

Convert the Type and Content Rating columns to a numeric format, whether by one-hot encoding, ordinal encoding, or similar.

In [11]:
# answer below:
content_ratings = {"Everyone": 1, "Teen": 2, "Mature 17+": 4, "Everyone 10+": 3}
reviews["Content Rating"] = reviews["Content Rating"].replace(
    to_replace=content_ratings
)



<IPython.core.display.Javascript object>

In [12]:
reviews = pd.get_dummies(reviews, columns=["Type"])
reviews = reviews.drop(columns=["Type_Free"])

<IPython.core.display.Javascript object>

In [13]:
reviews.head()

,Rating,Reviews,Price,Content Rating,Type_Paid
0,4.1,159,0,1,0
1,3.9,967,0,1,0
2,4.7,87510,0,1,0
3,4.5,215644,0,2,0
4,4.3,967,0,1,0


<IPython.core.display.Javascript object>

Finally, check that all the columns are of numeric type and change the type of columns that are not numeric. If coercing to numeric causes missing values, remove those rows containing missing values from our dataset.

In [14]:
# answer below:
reviews["Price"] = reviews["Price"].str.replace("$", "").astype("float")

<IPython.core.display.Javascript object>

In [15]:
reviews["Reviews"] = reviews["Reviews"].astype("int")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9362 entries, 0 to 10840
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Rating          9362 non-null   float64
 1   Reviews         9362 non-null   int32  
 2   Price           9362 non-null   float64
 3   Content Rating  9362 non-null   int64  
 4   Type_Paid       9362 non-null   uint8  
dtypes: float64(2), int32(1), int64(1), uint8(1)
memory usage: 338.3 KB


<IPython.core.display.Javascript object>

Perform a train test split with 20% of the data in the test sample.

In [16]:
# answer below:
X = reviews.drop(columns=["Rating"])
y = reviews["Rating"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

<IPython.core.display.Javascript object>

Now generate a linear model using statsmodels and produce a p value for each coefficient in the model. Analyze the results. (Look at the results table and at a homoscedasticity plot.)

In [17]:
# answer below:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model_1 = sm.OLS(y_train, X_train_const).fit()

<IPython.core.display.Javascript object>

In [18]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     15.82
Date:                Tue, 01 Sep 2020   Prob (F-statistic):           6.72e-13
Time:                        14:24:19   Log-Likelihood:                -5619.3
No. Observations:                7489   AIC:                         1.125e+04
Df Residuals:                    7484   BIC:                         1.128e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.1769      0.012    348.482      0.000       4.153       4.200
Reviews         1.265e-08   1.95e-09      6.502      0.000    8.84e-09    1.65e-08
Price             -0.0011      0.000     -2.657      0.008      -0.002      -0.000
Content Rating     0.0010      0.008      0.131      0.895      -0.014       0.016
Type_Paid          0.1061      0.024      4.469      0.000       0.060       0.153
==============================================================================
Omnibus:                     2922.142   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14435.857
Skew:                          -1.837   Prob(JB):                         0.00
Kurtosis:                       8.723   Cond. No.                     1.25e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.25e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>

Scale your predictors and refit the linear model.

* How does this change the coefficients?
* How does this change the coefficients' p values?
* How does this change model performance?

In [19]:
# answer below:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

<IPython.core.display.Javascript object>

In [20]:
X_scaled

array([[-0.16349082, -0.06077072, -0.44698887, -0.27246984],
       [-0.16323386, -0.06077072, -0.44698887, -0.27246984],
       [-0.13571209, -0.06077072, -0.44698887, -0.27246984],
       ...,
       [-0.16354011, -0.06077072, -0.44698887, -0.27246984],
       [-0.16350513, -0.06077072,  3.40279019, -0.27246984],
       [-0.03687473, -0.06077072, -0.44698887, -0.27246984]])

<IPython.core.display.Javascript object>

In [21]:
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

X_train_scaled_const = sm.add_constant(X_train_scaled)
X_test_scaled_const = sm.add_constant(X_test_scaled)

model_2 = sm.OLS(y_train, X_train_scaled_const).fit()

<IPython.core.display.Javascript object>

In [22]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     15.82
Date:                Tue, 01 Sep 2020   Prob (F-statistic):           6.72e-13
Time:                        14:24:19   Log-Likelihood:                -5619.3
No. Observations:                7489   AIC:                         1.125e+04
Df Residuals:                    7484   BIC:                         1.128e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1910      0.006    707.516      0.000       4.179       4.203
x1             0.0398      0.006      6.502      0.000       0.028       0.052
x2            -0.0174      0.007     -2.657      0.008      -0.030      -0.005
x3             0.0008      0.006      0.131      0.895      -0.011       0.012
x4             0.0269      0.006      4.469      0.000       0.015       0.039
==============================================================================
Omnibus:                     2922.142   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14435.857
Skew:                          -1.837   Prob(JB):                         0.00
Kurtosis:                       8.723   Cond. No.                         1.28
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>